# SQL

In [ ]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [ ]:
# Note that this is not the same db we have used in course videos, please download from this link
# https://drive.google.com/file/d/1O-1-L1DdNxEK6O6nG2jS31MbrMh-OnXM/view?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
filepath = '/content/drive/My Drive/Colab Notebooks/Assignments/As18_SQL Assignment on IMDB data/'

conn = sqlite3.connect(filepath + "IMDB_data.db")

#### Overview of all tables

In [ ]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [ ]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [ ]:
q1 = """
     SELECT p.Name Director, m.title Movie, CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) Year_cln
     FROM movie m
     JOIN m_genre gm ON m.MID = gm.MID
     JOIN genre g ON gm.GID = g.GID
     JOIN m_director dm ON m.MID = dm.MID
     JOIN person p ON dm.PID = p.PID
     WHERE (Year_cln % 4 == 0 AND Year_cln % 100 != 0 OR Year_cln % 400 ==0)
     AND g.Name Like '%Comedy%';
     """

df = pd.read_sql_query(q1, conn)
df

,Director,Movie,Year_cln
0,Milap Zaveri,Mastizaade,2016
1,Danny Leiner,Harold & Kumar Go to White Castle,2004
2,Anurag Kashyap,Gangs of Wasseypur,2012
3,Frank Coraci,Around the World in 80 Days,2004
4,Griffin Dunne,The Accidental Husband,2008
...,...,...,...
227,Siddharth Anand Kumar,Let's Enjoy,2004
228,Amma Rajasekhar,Sathyam,2008
229,Oliver Paulus,Tandoori Love,2008
230,Raja Chanda,Le Halua Le,2012


In [ ]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

grader_1(q1)

            Director                              Movie  Year_cln
0       Milap Zaveri                         Mastizaade      2016
1       Danny Leiner  Harold & Kumar Go to White Castle      2004
2     Anurag Kashyap                 Gangs of Wasseypur      2012
3       Frank Coraci        Around the World in 80 Days      2004
4      Griffin Dunne             The Accidental Husband      2008
5        Anurag Basu                             Barfi!      2012
6    Gurinder Chadha                  Bride & Prejudice      2004
7         Mike Judge    Beavis and Butt-Head Do America      1996
8   Tarun Mansukhani                            Dostana      2008
9       Shakun Batra                      Kapoor & Sons      2016
CPU times: user 64.9 ms, sys: 230 µs, total: 65.1 ms
Wall time: 65.8 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [ ]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))

query2 = """
         SELECT p.Name FROM Person p
         WHERE p.PID IN
         (SELECT TRIM(cm.PID) FROM movie m
         JOIN M_Cast cm ON m.MID = TRIM(cm.MID)
         WHERE title = 'Anand');
         """

grader_2(query2)

                Name
0   Amitabh Bachchan
1      Rajesh Khanna
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
5     Asit Kumar Sen
6         Dev Kishan
7       Atam Prakash
8      Lalita Kumari
9             Savita
CPU times: user 30.5 ms, sys: 853 µs, total: 31.4 ms
Wall time: 36.2 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [ ]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 ="""
Select p.PID from Person p
inner join
(
    select trim(mc.PID) PD, mc.MID from M_cast mc
where mc.MID
in
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)<1970
)
) r1
on r1.PD=p.PID
"""
query_more_1990 ="""
Select p.PID from Person p
inner join
(
    select trim(mc.PID) PD, mc.MID from M_cast mc
where mc.MID
in
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)>1990
)
) r1
on r1.PD=p.PID """
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question

(4942, 1)
(62570, 1)
True
CPU times: user 258 ms, sys: 11.3 ms, total: 270 ms
Wall time: 277 ms


In [ ]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """
         SELECT DISTINCT p.Name FROM Person p
         JOIN
         (
            SELECT TRIM(cm.PID) cmPD FROM M_Cast cm
            WHERE cm.MID IN
         (
            SELECT m.MID FROM Movie m
            WHERE CAST(SUBSTR(m.year,-4) AS Integer)<1970
         )
         ) sq
         ON p.PID = sq.cmPD
         WHERE p.PID IN
         (
            SELECT TRIM(cm.PID) cmPD FROM M_Cast cm
            WHERE cm.MID IN
         (
            SELECT m.MID FROM Movie m
            WHERE CAST(SUBSTR(m.year,-4) AS Integer)>1990
         )
         );
         """
grader_3(query3)

              Name
0   Waheeda Rehman
1    Johnny Walker
2          Mehmood
3            Ratna
4   Rajendra Kumar
5         Iftekhar
6        Raj Mehra
7     Lalita Pawar
8   Achala Sachdev
9       Sunil Dutt
CPU times: user 166 ms, sys: 3.07 ms, total: 169 ms
Wall time: 169 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [ ]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10))
    return (query_4a.shape == (1462,2))

query_4a = """
           SELECT md.PID Director_ID, count(*) Movie_Count FROM M_Director md
           GROUP BY md.PID
           """
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

  Director_ID  Movie_Count
0   nm0000180            1
1   nm0000187            1
2   nm0000229            1
3   nm0000269            1
4   nm0000386            1
5   nm0000487            2
6   nm0000965            1
7   nm0001060            1
8   nm0001162            1
9   nm0001241            1
True
CPU times: user 11.6 ms, sys: 945 µs, total: 12.5 ms
Wall time: 12.9 ms


In [ ]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 =  """
          SELECT p.Name, count(*) Movie_Count FROM M_Director md
          JOIN Person p ON md.PID = p.PID
          GROUP BY md.PID
          HAVING Movie_Count>=10
          ORDER BY Movie_Count DESC
          ;
          """
grader_4(query4)

                    Name  Movie_Count
0           David Dhawan           39
1           Mahesh Bhatt           35
2           Priyadarshan           30
3        Ram Gopal Varma           30
4           Vikram Bhatt           29
5   Hrishikesh Mukherjee           27
6            Yash Chopra           21
7        Basu Chatterjee           19
8         Shakti Samanta           19
9           Subhash Ghai           18
CPU times: user 56.7 ms, sys: 963 µs, total: 57.6 ms
Wall time: 58.3 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [ ]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10))
    return (query_5aa.shape == (8846,3))

query_5aa = """
            SELECT mc.MID, p.Gender, COUNT(*) AS COUNT FROM M_Cast mc
            JOIN Person p
            ON TRIM(mc.PID) = p.PID
            GROUP BY mc.MID, p.Gender
            """

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10))
    return (query_5ab.shape == (3469, 3))

query_5ab = """
            SELECT mc.MID, p.Gender, COUNT(*) AS COUNT FROM M_Cast mc
            JOIN Person p
            ON TRIM(mc.PID) = p.PID
            GROUP BY mc.MID, p.Gender
            HAVING p.Gender = 'Male'
            """

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

         MID  Gender  COUNT
0  tt0021594    None      1
1  tt0021594  Female      3
2  tt0021594    Male      5
3  tt0026274    None      2
4  tt0026274  Female     11
5  tt0026274    Male      9
6  tt0027256    None      2
7  tt0027256  Female      5
8  tt0027256    Male      8
9  tt0028217  Female      3
True
         MID Gender  COUNT
0  tt0021594   Male      5
1  tt0026274   Male      9
2  tt0027256   Male      8
3  tt0028217   Male      7
4  tt0031580   Male     27
5  tt0033616   Male     46
6  tt0036077   Male     11
7  tt0038491   Male      7
8  tt0039654   Male      6
9  tt0040067   Male     10
True
CPU times: user 313 ms, sys: 5.96 ms, total: 319 ms
Wall time: 320 ms


In [ ]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))

query5a = """
          SELECT CAST(SUBSTR(m.year, -4) AS INTEGER) Year_cln, COUNT(*) Female_Cast_Only_Movies FROM Movie m
          WHERE m.MID NOT IN
          (
          SELECT mc.MID FROM M_Cast mc
          JOIN Person p
          ON TRIM(mc.PID) = p.PID
          GROUP BY mc.MID, p.Gender
          HAVING p.Gender = 'Male'
          )
          GROUP BY Year_cln
          """
grader_5a(query5a)

   Year_cln  Female_Cast_Only_Movies
0      1939                        1
1      1999                        1
2      2000                        1
3      2018                        1
CPU times: user 121 ms, sys: 5.07 ms, total: 126 ms
Wall time: 126 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [ ]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = """
          SELECT CAST(SUBSTR(m.year, -4) AS INTEGER) Year_cln,
          CAST(sq.Female_Cast_Only_Movies AS FLOAT) *100 / COUNT(*) AS Percentage_Female_Only_Movies,
          COUNT(*) Movie_Count
          FROM Movie m
          JOIN
          (
            SELECT CAST(SUBSTR(m.year, -4) AS INTEGER) Year_cl, COUNT(*) Female_Cast_Only_Movies
            FROM Movie m
            WHERE m.MID NOT IN
            (
              SELECT mc.MID FROM M_Cast mc
              JOIN Person p
              ON TRIM(mc.PID) = p.PID
              GROUP BY mc.MID, p.Gender
              HAVING p.Gender = 'Male'
            )
            GROUP BY Year_cl
          ) sq
          ON Year_cln = sq.Year_cl
          GROUP BY Year_cln
          """

grader_5b(query5b)

   Year_cln  Percentage_Female_Only_Movies  Movie_Count
0      1939                      50.000000            2
1      1999                       1.515152           66
2      2000                       1.562500           64
3      2018                       0.961538          104
CPU times: user 118 ms, sys: 3.18 ms, total: 122 ms
Wall time: 123 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [ ]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 =  """
          SELECT m.title, COUNT(*) cast_size
          FROM M_Cast mc
          JOIN Movie m
          ON TRIM(mc.MID) = m.MID
          GROUP BY TRIM(mc.MID)
          ORDER BY Cast_Size DESC
          """
grader_6(query6)

                        title  cast_size
0               Ocean's Eight        238
1                    Apaharan        233
2                        Gold        215
3             My Name Is Khan        213
4  Captain America: Civil War        191
5                    Geostorm        170
6                     Striker        165
7                        2012        154
8                      Pixels        144
9       Yamla Pagla Deewana 2        140
CPU times: user 101 ms, sys: 3.98 ms, total: 105 ms
Wall time: 104 ms


### Q7 --- A decade is a sequence of 10 consecutive years.
### For example, say in your database you have movie information starting from 1931.
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on.
### Find the decade D with the largest number of films and the total number of films in D

In [ ]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

query7a = """
          SELECT CAST(SUBSTR(m.year, -4) AS INTEGER) Movie_Year, COUNT(*) Total_Movies
          FROM Movie m
          GROUP BY Movie_Year
          """
grader_7a(query7a)

# using the above query, you can write the answer to the given question

   Movie_Year  Total_Movies
0        1931             1
1        1936             3
2        1939             2
3        1941             1
4        1943             1
5        1946             2
6        1947             2
7        1948             3
8        1949             3
9        1950             2
CPU times: user 9.31 ms, sys: 921 µs, total: 10.2 ms
Wall time: 9.74 ms


In [ ]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

query7b = """
          SELECT Movie_Year, Movie_Count,
                 CAST(SUBSTR(m.year, -4) AS INTEGER) M_Movie_Year,
                 COUNT(*) M_Movie_Count FROM
          (
              SELECT CAST(SUBSTR(year, -4) AS INTEGER) Movie_Year, COUNT(*) Movie_Count
              FROM Movie
              GROUP BY Movie_Year
          ) sq
          JOIN Movie m
          ON M_Movie_Year >= sq.Movie_Year
          AND M_Movie_Year <= sq.Movie_Year+9
          GROUP BY sq.Movie_Year, M_Movie_Year
          ;
          """
grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

   Movie_Year  Movie_Count  M_Movie_Year  M_Movie_Count
0        1931            1          1931              1
1        1931            1          1936              3
2        1931            1          1939              2
3        1936            3          1936              3
4        1936            3          1939              2
5        1936            3          1941              1
6        1936            3          1943              1
7        1939            2          1939              2
8        1939            2          1941              1
9        1939            2          1943              1
CPU times: user 94 ms, sys: 1.01 ms, total: 95 ms
Wall time: 96.4 ms


In [ ]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 =  """
          SELECT Movie_Year Decade_start, SUM(M_Movie_Count) Decade_Movie_Count FROM
          (
              SELECT Movie_Year, Movie_Count,
              CAST(SUBSTR(m.year, -4) AS INTEGER) M_Movie_Year,
              COUNT(*) M_Movie_Count FROM
              (
                  SELECT CAST(SUBSTR(year, -4) AS INTEGER) Movie_Year, COUNT(*) Movie_Count
                  FROM Movie
                  GROUP BY Movie_Year
              ) sq
              JOIN Movie m
              ON M_Movie_Year >= sq.Movie_Year
              AND M_Movie_Year <= sq.Movie_Year+9
              GROUP BY sq.Movie_Year, M_Movie_Year
          )
          GROUP BY Decade_start
          ORDER BY Decade_Movie_Count DESC
          LIMIT 1
          ;
          """
grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

   Decade_start  Decade_Movie_Count
0          2008                1203
CPU times: user 95 ms, sys: 983 µs, total: 96 ms
Wall time: 99 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [ ]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = """
          SELECT TRIM(mc.PID) Actor_ID, md.PID Director_ID, COUNT(*) Movie_Count
          FROM M_Director md
          JOIN M_Cast mc
          ON md.MID = TRIM(mc.MID)
          GROUP BY Actor_ID, Director_ID
          """
grader_8a(query8a)

# using the above query, you can write the answer to the given question

    Actor_ID Director_ID  Movie_Count
0  nm0000002   nm0496746            1
1  nm0000027   nm0000180            1
2  nm0000039   nm0896533            1
3  nm0000042   nm0896533            1
4  nm0000047   nm0004292            1
5  nm0000073   nm0485943            1
6  nm0000076   nm0000229            1
7  nm0000092   nm0178997            1
8  nm0000093   nm0000269            1
9  nm0000096   nm0113819            1
CPU times: user 269 ms, sys: 11 ms, total: 280 ms
Wall time: 286 ms


In [ ]:
%%time
q8 =  """
      SELECT TRIM(p.Name) Actor, Max_Count FROM
      (
      SELECT TRIM(mc.PID) Actor_ID, md.PID Director_ID, COUNT(*) Movie_Count
      FROM M_Director md
      JOIN M_Cast mc
      ON md.MID = TRIM(mc.MID)
      GROUP BY Actor_ID, Director_ID
      ) class
      JOIN
      (
      SELECT Actor_ID, MAX(Movie_Count) Max_Count FROM
        (
          SELECT TRIM(mc.PID) Actor_ID, md.PID Director_ID, COUNT(*) Movie_Count
          FROM M_Director md
          JOIN M_Cast mc
          ON md.MID = TRIM(mc.MID)
          GROUP BY Actor_ID, Director_ID
        )
        GROUP BY Actor_ID
      ) max_counts
      ON class.Actor_ID = max_counts.Actor_ID
      AND class.Movie_Count = max_counts.Max_Count
      JOIN Person p ON class.Actor_ID = p.PID
      WHERE Director_ID = (SELECT PID FROM Person WHERE TRIM(Name) = 'Yash Chopra')
      ORDER BY Max_Count DESC
      """
df= pd.read_sql_query(q8, conn)
df

CPU times: user 438 ms, sys: 20.7 ms, total: 459 ms
Wall time: 465 ms


In [ ]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

# https://tableplus.com/blog/2018/05/mysql-select-row-max-value-distinct-bu-other-column.html
query8 =  """
          SELECT TRIM(p.Name) Actor, Max_Count FROM
          (
              SELECT TRIM(mc.PID) Actor_ID, md.PID Director_ID, COUNT(*) Movie_Count
              FROM M_Director md
              JOIN M_Cast mc
              ON md.MID = TRIM(mc.MID)
              GROUP BY Actor_ID, Director_ID
          ) class
          JOIN
          (
              SELECT Actor_ID, MAX(Movie_Count) Max_Count FROM
              (
                  SELECT TRIM(mc.PID) Actor_ID, md.PID Director_ID, COUNT(*) Movie_Count
                  FROM M_Director md
                  JOIN M_Cast mc
                  ON md.MID = TRIM(mc.MID)
                  GROUP BY Actor_ID, Director_ID
              )
              GROUP BY Actor_ID
          ) max_counts
          ON class.Actor_ID = max_counts.Actor_ID
          AND class.Movie_Count = max_counts.Max_Count
          JOIN Person p ON class.Actor_ID = p.PID
          WHERE Director_ID = (SELECT PID FROM Person WHERE TRIM(Name) = 'Yash Chopra')
          ORDER BY Max_Count DESC
          """
grader_8(query8)

              Actor  Max_Count
0       Jagdish Raj         11
1  Manmohan Krishna         10
2          Iftekhar          9
3     Shashi Kapoor          7
4     Rakhee Gulzar          5
5    Waheeda Rehman          5
6          Ravikant          4
7    Achala Sachdev          4
8       Neetu Singh          4
9     Leela Chitnis          3
(245, 2)
CPU times: user 462 ms, sys: 14.3 ms, total: 476 ms
Wall time: 472 ms


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [ ]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """
          SELECT DISTINCT TRIM(c.PID) S1_PID FROM M_Cast c
          WHERE TRIM(c.MID) IN
          (
              SELECT TRIM(mc.MID) Movie_ID FROM M_Cast mc
              WHERE TRIM(mc.PID) = (SELECT p.PID srk_ID FROM person p WHERE TRIM(p.Name) LIKE 'Shah Rukh%')
          )
          AND TRIM(c.PID) != (SELECT p.PID srk_ID FROM person p WHERE TRIM(p.Name) LIKE 'Shah Rukh%')
          """
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

      S1_PID
0  nm0004418
1  nm1995953
2  nm2778261
3  nm0631373
4  nm0241935
5  nm0792116
6  nm1300111
7  nm0196375
8  nm1464837
9  nm2868019
(2382, 1)
CPU times: user 36.5 ms, sys: 962 µs, total: 37.5 ms
Wall time: 37.7 ms


In [ ]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 =  """
          SELECT p.Name FROM Person p
          WHERE p.PID IN
          (
            SELECT DISTINCT TRIM(ca.PID) S2_PID FROM M_Cast ca
            WHERE TRIM(ca.MID) IN
            (
              SELECT DISTINCT TRIM(cm.MID) S1_Movie_IDs FROM M_Cast cm
              WHERE TRIM(cm.PID) IN
              (
                SELECT DISTINCT TRIM(c.PID) S1_PID FROM M_Cast c
                WHERE TRIM(c.MID) IN
                (
                  SELECT TRIM(mc.MID) Movie_ID FROM M_Cast mc
                  WHERE TRIM(mc.PID) = (SELECT p.PID FROM person p WHERE TRIM(p.Name) LIKE 'Shah Rukh%')
                )
                AND S1_PID != (SELECT p.PID FROM person p WHERE TRIM(p.Name) LIKE 'Shah Rukh%')
              )
            )
            AND S2_PID NOT IN (SELECT DISTINCT TRIM(c.PID) S1_PID FROM M_Cast c
                               WHERE TRIM(c.MID) IN
                               (
                                  SELECT TRIM(mc.MID) Movie_ID FROM M_Cast mc
                                  WHERE TRIM(mc.PID) = (SELECT p.PID FROM person p WHERE TRIM(p.Name) LIKE 'Shah Rukh%')
                               ))
          )
          """
grader_9(query9)

                     Name
0            Freida Pinto
1             Rohan Chand
2            Damian Young
3         Waris Ahluwalia
4   Caroline Christl Long
5           Rajeev Pahuja
6       Michelle Santiago
7         Alicia Vikander
8            Dominic West
9          Walton Goggins
(25698, 1)
CPU times: user 267 ms, sys: 4.93 ms, total: 272 ms
Wall time: 271 ms
